In [1]:
import pandas as pd

# read_file = pd.read_excel (r'data\addresses.xlsx', sheet_name='Result 1')
# read_file.to_csv (r'data\addresses.csv', index = None, header=True)

In [2]:
read_file = pd.read_csv (r'data\addresses.csv')

In [3]:
read_file

,address,building_no,unit
0,ستاری بالاتر مجتمع کوروش کوچه اسدی,24,2
1,میدان کاج به طرف سرو غربی میدان قیصر امین پور ...,39,10
2,پونک/بلوار میرزابابایی/خیابان ایران زمین شمالی...,30,3
3,تهران،فرودگاه مهرآباد,7,8
4,تهران،خ. باهنر، نرسیده به سه راه یاسر، خ. صادق...,2,3
...,...,...,...
199995,تهران،پیروزی، دهقان، خ. دهقان، خ. تقی قاسمیان,55,2
199996,تهران،پیروزی، خ. زینتی افخم، بعد از خ. درودیان...,14,3
199997,تهران،پیروزی، خ. پرستار، نرسیده به خ. برادران ...,2,همکف
199998,تهران،محلاتی، خ. قیام، نرسیده به خ. پاسدار گمن...,52,3


In [29]:
'\u0627'

'ا'

In [4]:
class PersianNormalizer:
    def __init__(self):
        
        self.switcher = {
            'ALEF' : '\u0627',
            'ALEF_MADDA' : '\u0622',
            'ALEF_HAMZA_ABOVE' : '\u0623',
            'ALEF_HAMZA_BELOW' : '\u0625',

            'WAW' : '\u0648',
            'WAW_HAMZA_ABOVE' : '\u0624',

            'YEH' : '\u064A',
            'YEH_HAMZA_ABOVE' : '\u0626',
            'FARSI_YEH' : '\u06CC',

            'KAF' : '\u0643',
            'KEHEH' : '\u06A9',

            'HEH' : '\u0647',
            'HEH_HAMZA_ABOVE' : '\u06C2',
            'TEH_MARBUTA' : '\u0629',

            'KASHIDA' : '\u0640',

            'HAMZA' : '\u0621',

            'FATHATAN' : '\u064B',
            'DAMMATAN' : '\u064C',
            'KASRATAN' : '\u064D',
            'FATHA' : '\u064E',
            'DAMMA' : '\u064F',
            'KASRA' : '\u0650',
            'SHADDA' : '\u0651',
            'SUKUN' : '\u0652'
        }
        
        self.switche = {}
        for (k, v) in self.switcher.items():
            v_new = v
            if k in ['ALEF_MADDA', 'ALEF_HAMZA_ABOVE', 'ALEF_HAMZA_BELOW']:
                v_new = self.switcher['ALEF']
            elif k in ['WAW_HAMZA_ABOVE']:
                v_new = self.switcher['WAW']
            elif k in ['YEH', 'YEH_HAMZA_ABOVE']:
                v_new = self.switcher['FARSI_YEH']
            elif k in ['KAF']:
                v_new = self.switcher['KEHEH']
            elif k in ['TEH_MARBUTA', 'HEH_HAMZA_ABOVE']:
                v_new = self.switcher['HEH']
            elif k in ['KASHIDA', 'KASRATAN', 'DAMMATAN', 'FATHATAN', 'FATHA', 'DAMMA', 'KASRA', 'SHADDA', 'SUKUN', 'HAMZA']:
                v_new = ''
            self.switche[v] = v_new
        
        

    def normalize(self, text):

        stringBuilder = ""
        for i in range(len(text)):
            if text[i] in self.switche.keys():
                stringBuilder += self.switche.get(text[i])
            else:
                stringBuilder += text[i]
            
        return stringBuilder
    

In [27]:
synonyms = {
    'خ': 'خیابان',
    'خیابان': 'خیابان',
    'ک': 'کوچه',
    'کوچه': 'کوچه',
    'ب': 'بلوار',
    'بلوار': 'بلوار',
    'بل': 'بلوار',
    'پ': 'پلاک',
    'پلاک': 'پلاک',
    'م': 'میدان',
    'میدان': 'میدان',
    'بن': 'بن‌بست',
    'بن‌بست': 'بن‌بست',
    'بن بست': 'بن‌بست'
}

In [30]:
# for i in range(ord('\u0600'), ord('\u06FF') + 1):
#     print(chr(i))

In [134]:
import re

def partition(arr, split_char):
    res = []
    for s in arr:
        s = s.strip()
        if s == '':
            continue
        s1 = []
        k = s.find(split_char)
        s2 = ''
        while k != -1:
            flag = True
            if k > 0 and (ord(s[k-1]) <= ord('ی') and ord(s[k-1]) >= ord('ا')):
                flag = False
            if k + len(split_char) < len(s) and (ord(s[k+len(split_char)]) <= ord('ی') and ord(s[k+len(split_char)]) >= ord('ا')):
                flag = False
            if flag:
                s2 += s[:k]
                if s2 != '':
                    s1.append(s2)
                s2 = synonyms[split_char]
            else:
                s2 += s[:k+len(split_char)]
            s = s[k+len(split_char):]
            k = s.find(split_char)
        s2 += s
        if s2 != '':
            s1.append(s2)
        res += s1
        
    return res
        

                  
def f(s):
#     res = re.sub(r'[^\w\s]', ' ', s)
    res = PersianNormalizer().normalize(s)
    res = re.split('\- |\, |\/ |\،', res)
    for i in synonyms.keys():
        res = partition(res, i)
#     print(res)
    return res

seen = []

def f2(s):
    res_str = ''
    s1 = s
    s = s.split(" ")
    for w in s:
        w = w.strip()
        if len(w) == 2 and w != '' and w not in seen:
            seen.append(w)
            res_str += w + ' '
    return res_str

In [135]:
f(" سل- مو,   ي گخ ")

['سل', 'مو', 'ی گخ']

In [133]:
" k ".strip()

'k'

In [91]:
read_file["address"][2]

'پونک/بلوار میرزابابایی/خیابان ایران زمین شمالی/ک اعلایی3/'

In [136]:
addresses = read_file["address"].apply(lambda x: f(x)).to_frame()

In [137]:
addresses

,address
0,"[ستاری بالاتر مجتمع کوروش, کوچه اسدی]"
1,"[میدان کاج به طرف سرو غربی, میدان قیصر امین پو..."
2,"[پونک/, بلوار میرزابابایی/, خیابان ایران زمین ..."
3,"[تهران, فرودگاه مهراباد]"
4,"[تهران, خیابان. باهنر, نرسیده به سه راه یاسر, ..."
...,...
199995,"[تهران, پیروزی, دهقان, خیابان. دهقان, خیابان. ..."
199996,"[تهران, پیروزی, خیابان. زینتی افخم, بعد از, خی..."
199997,"[تهران, پیروزی, خیابان. پرستار, نرسیده به, خیا..."
199998,"[تهران, محلاتی, خیابان. قیام, نرسیده به, خیابا..."


In [138]:
addresses['address'].iloc[199999]

['تهران',
 'پیروزی',
 'خیابان. زینتی افخم',
 'نرسیده به',
 'میدان. سراسیاب دولاب',
 'کوچه. محمد عباسی']

In [62]:
a = read_file[read_file["address"].str.contains('ايت') & ~read_file["address"].str.contains('آیت الله')]

In [63]:
a

,address,building_no,unit
20650,تهران،نیاوران، دار آباد، خ. پور ايتهاج ،سه راه...,17,1
35051,تهران،شهرک محلاتی، خ. ولایت، خ ولايت سوم برج فراز,4,004
37672,خيابان دروس. خيابان كماسايي. كوچه غفاري (جنب ب...,20,دوم شرقي
37838,دروس خيابان هدايت كوچه جورابچي,16,9
37885,تهران،خ شريعتي، خ هدايت، خ. شیفته,6,3
38032,تهران،پاسداران، دروس، خ. هدايت ، خيابان حجت سوري,59,105
39002,تهران،دروس -خيابان هدايت خ. وارسته، خ. جورابچی,20,2
39004,تهران،قلهك دروس انتهاى خ. هدايت خ. حمید بهیار,27,1
39022,دروس، بلوار شهرزاد، خيابان هدايت، كوچه پنجم، پ...,21,..
41105,تهران،ميدان رسالت، خ. کرد ،خ.هدايتي، خ. خانعلی...,4,5


In [56]:
a.iloc[9]["address"]

'تهران،ولنجک، خ. مقدس اردبیلی، خ. ثارالله\nکافه بورس'

In [84]:
ab = addresses["address"].apply(lambda x: f2(x)).unique()

In [79]:
ab

array(['', 'ک ', 'خ ', 'ب م ', 'پ ', '1 ', 'و ', 'ی ۴ ', '4 ', '۲ ', '۸ ',
       '5 ', '9 ', 'ا ', '۳ ', '7 6 ', '۵ ۶ ', '۱ ', '2 ', '8 ', 'A ',
       '٢ ', 'ج ', '٣ ', '3 ', 'ك ', '٧ ', 'ي ', '۷ ', '۹ ', '_ ', '٦ ',
       '١ ', '٨ ', 'ط ', '٥ ', '٩ ', '٤ ', 'ز ', 'ق ', 'C ', 'D ', 'B ',
       'd ', 'ش ', 'ن ', 'ع ', 'د ', 'ح ', 'آ ', 'ى ', 'ر ', 'ﺥ ', '۰ ',
       'c ', 'ه ', 'E ', 'e ', 'G ', 'f ', 'F ', 'ص ', 'g ', 'ظ ', 'ف ',
       'b ', 'س ', 'چ ', 'a ', '0 ', 'ئ ', 'i ', 'N ', 'ـ ', 'ت ', 'گ ',
       'p ', 'غ ', 'ل ', 'L ', 'k ', 'T ', 'z ', 'M ', 'H ', 'm ', 'ء '],
      dtype=object)

In [85]:
ab

array(['', 'به سر 10 ', 'سه ', 'بل ', 'بن ', '11 ', 'نو ', 'از ', 'A3 ',
       '21 ', 'کن ', 'ری ', 'شش 24 ', 'دو ', '۳۱ ', 'رو ', 'سی ',
       'اب نه ', 'حر ', 'زر ۶۵ ', 'جی ', 'ده ', '47 ', 'با ', 'ای ',
       '۱۵ ', '34 ', 'حق ', 'گل ', '26 ', '۳۲ ', '۲۴ ', '۲۶ می ', 'ام ',
       '۱۶ ', '۱۲ ', '16 ', '12 ', 'در ', '۱۰ ', '۴۰ یا ', '۲۰ ۲۲ ',
       '١٩ ', '١٥ ۴۸ c2 ', 'یک ', '36 ', '۱۱ ', '۳۳ ', 'چپ ', '۱۴ ',
       '۲۳ ', '۷۵ ', 'دی ', '۲۹ ', 'پ۷ ', '18 ', '۱۹ ۱۷ ', '۳۹ ', 'یخ ',
       '۳_ ', '15 ', '19 ', 'رز ۶۶ ', '۵۰ ', '۳۵ ', '48 ', '١٢ ', '۱۳ ',
       '۲۷ ', '62 ', '13 ', 'تر 56 ', '۵۳ ', '۱۸ ', 'ط٥ ', 'آل ', '27 ',
       'من ', '14 ', '۵۴ ', '۷۴ ', '۹۷ ', '40 ', '77 ', '23 ', '54 ',
       '٤٤ ', '١٤ ', '٢٢ ', '41 ', '22 87 ', '30 ', '٤٠ ', 'B3 ', '۴۳ ',
       '۲۱ ', '٢٦ ', '28 ', '٤٦ تك ', 'پل ', 'ط۵ ', '۲۸ ', '۲۵ ', '۸۲ ',
       '60 ', '66 ', 'سو ', 'آب ', '٥٢ ', '17 ', '١٨ ', '۳۶ ', '٢٩ ',
       '20 ', '32 ط4 ', 'لو ', 'هم ', '۳۸ تک ', 'طب ', 'تا ٣٠ ', '۶۷ ',
    